# Use pre-searched policy

## Read configuration

In [1]:
import pkg_resources
from theconf import Config as C

configfile="./taa/confs/english_data.yaml"

"""use pre-searched policy to augment texts"""
if configfile is None:
    resource_package = __name__
    resource_path = '/'.join(('confs', 'bert_imdb_example.yaml'))
    configfile = pkg_resources.resource_filename(resource_package, resource_path)

_ = C(configfile)

In [2]:
name = C.get()['dataset']['name']
path = C.get()['dataset']['path']
data_dir = C.get()['dataset']['data_dir']
data_files = C.get()['dataset']['data_files']

In [3]:
from datasets import load_dataset # load huggingface dataset

if path is None:
    train_dataset = load_dataset(path=name, data_dir=data_dir, data_files=data_files, split='train')
else:
    train_dataset = load_dataset(path=path, name=name, data_dir=data_dir, data_files=data_files, split='train')

/home/tuan/anaconda3/envs/taa/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration data-2e781313747ead65
Found cached dataset csv (/home/tuan/.cache/huggingface/datasets/csv/data-2e781313747ead65/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


In [4]:
train_dataset = load_dataset(path="./taa/data", data_files={'train': 'english_data_train.csv', 'test': 'custom_data_test.csv'}, split='train')

Using custom data configuration data-696021b8a1128382
Found cached dataset csv (/home/tuan/.cache/huggingface/datasets/csv/data-696021b8a1128382/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


In [5]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 2
})

In [6]:
from taa.utils.train_tfidf import train_tfidf

train_tfidf("imdb")  # calculate tf-idf score for TS and TI operations

Use exist TF-IDF model


In [8]:
from taa.archive import policy_map

assert name in list(policy_map.keys()), "No policy was found for this dataset: " + name
policy = policy_map[name]
n_aug = C.get()['n_aug']
policy

[[('tfidf_word_substitute', 0.7701989738031709, 0.1413189624238933),
  ('tfidf_word_substitute', 0.5017627182356647, 0.6120126108801994)],
 [('random_word_delete', 0.723773044425342, 0.09343851538354664),
  ('tfidf_word_insert', 0.65871023661296, 0.18515171972877364)],
 [('tfidf_word_insert', 0.6855258510775246, 0.25155717455902515),
  ('random_word_swap', 0.5855398958576279, 0.8038331622764382)],
 [('tfidf_word_substitute', 0.766621630113518, 0.9996972209388315),
  ('tfidf_word_substitute', 0.35757492431715654, 0.5094646924565402)],
 [('tfidf_word_insert', 0.7201447184878109, 0.27394476379211474),
  ('tfidf_word_insert', 0.7102010737993143, 0.7735198668033543)],
 [('random_word_swap', 0.2602481801512967, 0.1094817565926513),
  ('random_word_swap', 0.377657390084737, 0.5860711221450985)],
 [('random_word_swap', 0.3628204769855412, 0.4174830136614628),
  ('tfidf_word_substitute', 0.7449752005673833, 0.5806015818156878)],
 [('random_word_delete', 0.6869314076454281, 0.30212959898754665),

In [9]:
from taa.data import augment

augmented_train_dataset = augment(dataset=train_dataset, policy=policy, n_aug=n_aug)

['The sailors rode the breeze clear of the rocks.', 'The more you would want, the less you would eat.']
['The sailors rode the breeze clear of the rocks.', 'The more you would want, the less you would eat.']
label:  [1, 0]
[["The crewman ride the breeze open of the rock 'n' roll."], ['The sailors rode the than clear of the rocks.'], ['Sailors the rode clear breeze the the of rocks.'], ['The sailors rode the breeze clear this of the rocks.'], 'The more you would want, the less you would eat.', ['The more than you would desire, the less you would eat.'], 'The more you would want, the less you would eat.', 'The more you would want, the less you would eat.']


In [13]:
augmented_train_dataset.texts

[["The crewman ride the breeze open of the rock 'n' roll."],
 ['The sailors rode the than clear of the rocks.'],
 ['Sailors the rode clear breeze the the of rocks.'],
 ['The sailors rode the breeze clear this of the rocks.'],
 'The more you would want, the less you would eat.',
 ['The more than you would desire, the less you would eat.'],
 'The more you would want, the less you would eat.',
 'The more you would want, the less you would eat.']

In [14]:
augmented_train_dataset.labels

[1, 1, 1, 1, 0, 0, 0, 0]

In [32]:
from taa.transforms import Compose
from taa.data import Augmentation
from transformers import BertTokenizer, BertTokenizerFast
from taa.utils.raw_data_utils import get_examples


C.get()['n_aug'] = n_aug
text_key = C.get()['dataset']['text_key']


In [33]:
transform_train = Compose([])
transform_train.transforms.insert(0, Augmentation(policy))


In [34]:
transform_train(texts=["hello my name is tuan"], labels=[1])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

([['hello my name Betsy is tuan'],
  'hello my name is tuan',
  ['hello Over him France tuan'],
  ['hello Rosie name afraid tuan']],
 [1, 1, 1, 1],
 0.23809523809523808)

In [35]:

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

examples = get_examples(train_dataset, text_key)



In [36]:
train_dataset.num_rows

2

In [37]:
[ "{} - {} - {} - {}".format(e.guid, e.text_a, e.text_b, e.label) for e in examples[0:1]]

['0 - The sailors rode the breeze clear of the rocks. - None - 1']

In [45]:
from taa.custom_dataset import general_dataset
augmented_dataset = general_dataset(examples, tokenizer, text_transform=transform_train)

['The sailors rode the breeze clear of the rocks.', 'The more you would want, the less you would eat.']
['The sailors rode the breeze clear of the rocks.', 'The more you would want, the less you would eat.']
label:  [1, 0]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to av

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [25]:

texxx = ['Jenny sailors rode Himself breeze clear of Carla rocks.', 'The sailors rode the breeze marry clear of the rocks.', 'The rode sailors Drowning clear breeze Himself him. rocks', 'Sailors the rode breeze the clear the of rocks.']

In [26]:
tokenizer.__call__(texxx, max_length=128, padding='max_length', truncation=True)

{'input_ids': [[101, 8437, 11279, 8469, 2370, 9478, 3154, 1997, 17081, 5749, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1996, 11279, 8469, 1996, 9478, 5914, 3154, 1997, 1996, 5749, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1996, 8469, 11279, 14759, 3154, 9478, 2370, 2032, 1012, 5749, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [44]:
train_dataset.labels

AttributeError: 'Dataset' object has no attribute 'labels'

In [39]:
testt = [['The sailors rode the clear breeze the rocks of.'], ['The sailors would rode proposal the breeze afraid clear of the marry rocks.'], ['Him sailors grandfather find pastry clear of blond rocks.'], ['Sailors the the rode clear breeze the of rocks.'], ['Sheep the find more you abroad would want, Jenny the blond less you person very would explode eat.'], ['No more smile would Jenny, shocks their you would eat.'], 'The more you would want, the less you would eat.', 'The more you would want, the less you would eat.']


In [40]:
type(testt)

list

In [44]:
test2 = []
for i in testt:
    if type(i) is list:
        test2.extend(i)
    else:
        test2.append(i)

test2

['The sailors rode the clear breeze the rocks of.']
<class 'list'>
['The sailors would rode proposal the breeze afraid clear of the marry rocks.']
<class 'list'>
['Him sailors grandfather find pastry clear of blond rocks.']
<class 'list'>
['Sailors the the rode clear breeze the of rocks.']
<class 'list'>
['Sheep the find more you abroad would want, Jenny the blond less you person very would explode eat.']
<class 'list'>
['No more smile would Jenny, shocks their you would eat.']
<class 'list'>
The more you would want, the less you would eat.
<class 'str'>
The more you would want, the less you would eat.
<class 'str'>


['The sailors rode the clear breeze the rocks of.',
 'The sailors would rode proposal the breeze afraid clear of the marry rocks.',
 'Him sailors grandfather find pastry clear of blond rocks.',
 'Sailors the the rode clear breeze the of rocks.',
 'Sheep the find more you abroad would want, Jenny the blond less you person very would explode eat.',
 'No more smile would Jenny, shocks their you would eat.',
 'The more you would want, the less you would eat.',
 'The more you would want, the less you would eat.']

# Search policy for dataset

# Search policy for Vietnamese dataset